## Isias Incidental Take Summary 05/04/2018 Run

In [8]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import numpy as np
import os, glob

In [25]:
isias_file = '/Users/mflaxman/Downloads/isias_proj1_scen42_20180504/impacts.shp'
date = '05042018'

In [29]:
isias_run = gpd.read_file(isias_file)
isias_run.head()

,SCENARIO,SC_NAME,SC_VERSION,TIMESTEP,TS_START,TS_END,ACT_NAME,ACT_CODE,SPE_NAME,SPE_CODE,IMP_AREA,HAB_PATCH,IMPACT_ID,ACRES,QUALITY,geometry
0,42,Plan Trend with Directional Armoring,3,118,2022-01-01,2022-12-31,Direct - Beach Armoring Landward,ARMR,Seaturtles,STRT,28238205,5800,5124862,0.007989,5,"POLYGON ((-80.06443301772829 26.407274675558, ..."
1,42,Plan Trend with Directional Armoring,3,128,2032-01-01,2032-12-31,"Direct - Large Non-native Landscape, Pools & D...",LSPl,Seaturtles,STRT,28348489,5877,5149762,0.017754,3,"POLYGON ((-86.62616827478796 30.3971371011565,..."
2,42,Plan Trend with Directional Armoring,3,128,2032-01-01,2032-12-31,"Direct - Large Non-native Landscape, Pools & D...",LSPl,Seaturtles,STRT,28348495,5877,5149764,0.023580,3,POLYGON ((-87.17867281770512 30.32551701167347...
3,42,Plan Trend with Directional Armoring,3,128,2032-01-01,2032-12-31,"Direct - Large Non-native Landscape, Pools & D...",LSPl,Seaturtles,STRT,28348496,5877,5149765,0.053897,3,"POLYGON ((-87.17641436095265 30.3259034860321,..."
4,42,Plan Trend with Directional Armoring,3,134,2038-01-01,2038-12-31,Direct - Beach Armoring Landward,ARMR,Seaturtles,STRT,28345686,5806,5152396,0.006175,5,POLYGON ((-82.26676814599895 26.77115517152771...


#### Strip Armoring Calculations from ISIAS run (because final method was directional)

In [61]:
dup = 'Direct - Beach Armoring Landward'
isias_run.drop(isias_run[isias_run.ACT_NAME == dup].index, inplace=True)
dup = 'Indirect - Armoring'
isias_run.drop(isias_run[isias_run.ACT_NAME == dup].index, inplace=True)
# SPE_NAME
dup = 'Gopher Tortoise'
isias_run.drop(isias_run[isias_run.SPE_NAME == dup].index, inplace=True)

In [62]:
isias_run.ACT_NAME.unique()

array([u'Direct - Large Non-native Landscape, Pools & Decks',
       u'Indirect - Single Family Activity', u'Indirect - Fill permits',
       u'Direct - Small Non-native Landscape, Pools & Decks',
       u'Direct - Dune Crossovers',
       u'Indirect - Multifamily/Commercial/Institutional Activities',
       u'Indirect - Dune Crossovers (SFR)',
       u'Direct - Building Footprint - SFR',
       u'Direct - Building Footprint - MFR/COM'], dtype=object)

In [63]:
# rename internal attributes
isias_run['ACT_NAME'] = isias_run['ACT_NAME'].replace(to_replace='Medium ', value='', regex=True)
isias_run['ACT_NAME'] = isias_run['ACT_NAME'].replace(to_replace='Direct - Building Footprint - Small', value='Direct - Building Footprint - SFR', regex=True)
isias_run['ACT_NAME'] = isias_run['ACT_NAME'].replace(to_replace='Direct - Building Footprint - Large', value='Direct - Building Footprint - MFR/COM', regex=True)

In [64]:
isias_run.sort_values(by='ACT_NAME').head()

,SCENARIO,SC_NAME,SC_VERSION,TIMESTEP,TS_START,TS_END,ACT_NAME,ACT_CODE,SPE_NAME,SPE_CODE,IMP_AREA,HAB_PATCH,IMPACT_ID,ACRES,QUALITY,geometry
48851,42,Plan Trend with Directional Armoring,3,123,2027-01-01,2027-12-31,Direct - Building Footprint - MFR/COM,FTPl,Seaturtles,STRT,28250951,5857,5132382,0.326012,5,"POLYGON ((-81.144217197264 29.51540336837166, ..."
55136,42,Plan Trend with Directional Armoring,3,128,2032-01-01,2032-12-31,Direct - Building Footprint - MFR/COM,FTPl,Beach Mice,BMSE,28355552,6246,5145929,0.482078,3,POLYGON ((-87.44196562405563 30.29472941792676...
3295,42,Plan Trend with Directional Armoring,3,129,2033-01-01,2033-12-31,Direct - Building Footprint - MFR/COM,FTPl,Beach Mice,BMSE,28255434,6092,5134143,0.222188,3,POLYGON ((-86.87838781873661 30.37768632357855...
54554,42,Plan Trend with Directional Armoring,3,122,2026-01-01,2026-12-31,Direct - Building Footprint - MFR/COM,FTPl,Seaturtles,STRT,28342229,5857,5144741,0.128587,5,POLYGON ((-81.14949287736574 29.52600779269519...
29026,42,Plan Trend with Directional Armoring,3,116,2020-01-01,2020-12-31,Direct - Building Footprint - MFR/COM,FTPl,Nonbreeding Shorebirds,NBSB,28185489,5710,5099875,0.383488,1,"POLYGON ((-84.9086519955892 29.63954853812608,..."


In [65]:
outfile = 'intermediates/gdt_upland_impacts_{}'.format(date)
isias_run.to_file(outfile)

#### Copy Final Files to Final Location, Adding Metadata to Filenames

In [57]:
def copy_shapefile(oldpath, newpath):
    required_count = 3
    actual_count = 0
    cwd = %pwd

    oldbase = os.path.basename(oldpath).replace('.shp','')
    #print('oldbase is {}'.format(oldbase))
    oldpath = oldpath.replace(os.path.basename(oldpath), '')
    #print('oldpath is {}'.format(oldpath))
    #if path isn't absolute, starting with a slash, then treat as relative to pwd
    if newpath[1] != '/':
        newpath = os.path.join(cwd, newpath)
    #print('newpath is {}'.format(newpath))
    required_parts = ('shp', 'shx', 'dbf')
    optional_parts = ('prj', 'sbx', 'sbn')
    
    for p in required_parts:
        oldfile = os.path.join(oldpath, '{}.{}'.format(oldbase, p))
        newfile = os.path.join(newpath, '{}.{}'.format(oldbase, p))
        try:
            !cp {oldfile} {newfile}
        except:
            print('Could not copy required shapefile part "{}"'.format(oldfile))
        actual_count += 1

    for p in optional_parts:
        oldfile = os.path.join(oldpath, '{}.{}'.format(oldbase, p))
        newfile = os.path.join(newpath, '{}.{}'.format(oldbase, p))
        if os.path.exists(oldfile):
            !cp {oldfile} {newfile}
            
    if actual_count == required_count:
        print('Successfully found and copied required elements: \n{} files copied to {}'.format(actual_count, newpath))

copy_shapefile(isias_file, 'intermediates')

Successfully found and copied required elements: 
3 files copied to /Users/mflaxman/Dropbox/fbhcp/notebooks/intermediates


In [36]:
def rename_shapefile(oldname, newname):
    required_parts = ('shp', 'shx', 'dbf')
    optional_parts = ('prj', 'sbx', 'sbn')
    cwd = %pwd

    oldbase = os.path.basename(oldname).replace('.shp','')
    print('oldbase is {}'.format(oldbase))
    oldname = oldname.replace(os.path.basename(oldname), '')
    print('oldname is {}'.format(oldname))

    #if path isn't absolute, starting with a slash, then treat as relative to pwd
    if newname[1] != '/':
        newname = os.path.join(cwd, newname)

    print('newname is {}'.format(newname))
    for p in required_parts:
        oldpart = os.path.join(oldname, '{}.{}'.format(oldbase, p))
        newpart = os.path.join(newname, '{}.{}'.format(newname, p))
        command = 'mv {} {}'.format(oldpart, newpart)
        print(command) 
        !{command}

rename_shapefile('intermediates/impacts.shp','intermediates/isias_run_raw_data_{}'.format(date))

oldbase is impacts
oldname is intermediates/
newname is /Users/mflaxman/Dropbox/fbhcp/notebooks/intermediates/isias_run_raw_data_05042018
mv intermediates/impacts.shp /Users/mflaxman/Dropbox/fbhcp/notebooks/intermediates/isias_run_raw_data_05042018.shp
mv: rename intermediates/impacts.shp to /Users/mflaxman/Dropbox/fbhcp/notebooks/intermediates/isias_run_raw_data_05042018.shp: No such file or directory
mv intermediates/impacts.shx /Users/mflaxman/Dropbox/fbhcp/notebooks/intermediates/isias_run_raw_data_05042018.shx
mv: rename intermediates/impacts.shx to /Users/mflaxman/Dropbox/fbhcp/notebooks/intermediates/isias_run_raw_data_05042018.shx: No such file or directory
mv intermediates/impacts.dbf /Users/mflaxman/Dropbox/fbhcp/notebooks/intermediates/isias_run_raw_data_05042018.dbf
mv: rename intermediates/impacts.dbf to /Users/mflaxman/Dropbox/fbhcp/notebooks/intermediates/isias_run_raw_data_05042018.dbf: No such file or directory


In [28]:
!ls -l intermediates/gdt_upland*

-rw-r--r--@ 1 mflaxman  staff  35467093 Jul  9 09:43 intermediates/gdt_upland_impacts_05042018.dbf
-rw-r--r--@ 1 mflaxman  staff  28665236 Jul  9 09:43 intermediates/gdt_upland_impacts_05042018.shp
-rw-r--r--@ 1 mflaxman  staff    640580 Jul  9 09:43 intermediates/gdt_upland_impacts_05042018.shx
-rw-r--r--@ 1 mflaxman  staff  35467093 Jul  9 09:24 intermediates/gdt_upland_impacts_05082018.dbf
-rw-r--r--@ 1 mflaxman  staff  28665236 Jul  9 09:24 intermediates/gdt_upland_impacts_05082018.shp
-rw-r--r--@ 1 mflaxman  staff    640580 Jul  9 09:24 intermediates/gdt_upland_impacts_05082018.shx


### Export Isias file to Excel (loosing geographic info)

In [66]:
output = 'outputs/gdt_upland_impacts_{}.xls'.format(date)
isias_run.to_excel(output)

In [67]:
!open {output}